In [2]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
import torch
import scib_metrics
from lightning.pytorch.callbacks import ModelCheckpoint
import copy
%matplotlib inline
import sys
sys.path.append('..')
from functions import *
from SCVIModelCheckpoint import SCVIModelCheckpoint 

/local/data/scvi-tools/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/local/data/scvi-tools/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  dispatcher = jit(_target='npyufunc',
/local/data/anaconda3/envs/BT/lib/python3.11/site-pac

In [3]:
adata = anndata.read_h5ad(filename="../data/adataLung4.h5ad")

In [4]:
resultsSD = pd.read_csv("results/result_SD.csv")
resultsMG = pd.read_csv("results/result_MG.csv")
resultsVP = pd.read_csv("results/result_VP.csv")
resultsNF = pd.read_csv("results/result_NF.csv")

In [5]:
def computeMeanStandardDeviation(results, prior, n):
    resultsMSD = results[["Unnamed: 0"]]
    resultsMSD["NMI Mean"] = np.zeros(39)
    resultsMSD["ARI Mean"] = np.zeros(39)
    resultsMSD["KBET Mean"] = np.zeros(39)
    for i in range(5):
        resultsMSD["NMI Mean"] += results[f"{prior}Leiden NMI{i}"]
        resultsMSD["ARI Mean"] += results[f"{prior}Leiden ARI{i}"]
        resultsMSD["KBET Mean"] += results[f"{prior}KBET{i}"]
    resultsMSD = resultsMSD * 1/n
    resultsMSD = resultsMSD.drop("Unnamed: 0",axis=1)
    resultsMSD["NMI SD"] = np.zeros(39)
    resultsMSD["ARI SD"] = np.zeros(39)
    resultsMSD["KBET SD"] = np.zeros(39)
    for i in range(5):
        resultsMSD["NMI SD"] += (results[f"{prior}Leiden NMI{i}"]-resultsMSD["NMI Mean"])**2
        resultsMSD["ARI SD"] += (results[f"{prior}Leiden ARI{i}"]-resultsMSD["ARI Mean"])**2
        resultsMSD["KBET SD"] += (results[f"{prior}KBET{i}"]-resultsMSD["KBET Mean"])**2
    resultsMSD[["NMI SD", "ARI SD","KBET SD"]] = (resultsMSD[["NMI SD", "ARI SD","KBET SD"]] * 1/(n-1))**0.5
    max_epochs = 390; n_eps = 10
    resultsMSD["index"] = np.linspace(n_eps-1,max_epochs-1, int(max_epochs/n_eps)).astype(np.int16)
    return resultsMSD

In [6]:
resultsSD_MSD = computeMeanStandardDeviation(resultsSD,"sdnormal",5)
resultsMG_MSD = computeMeanStandardDeviation(resultsMG,"mixofgaus",5)
resultsVP_MSD = computeMeanStandardDeviation(resultsVP,"vamp",5)
resultsNF_MSD = computeMeanStandardDeviation(resultsNF,"normalflow",5)

In [7]:
def plotEpochs(resultsSD, resultsMG, resultsVP, resultsNF):
    fig, axes = plt.subplots(1,3,figsize=(15,4))
    metrics = ["NMI Mean", "ARI Mean", "KBET Mean"]
    metrics2 = ["NMI","ARI","KBET"]
    for i in range(3):
        data = pd.DataFrame(np.array([resultsSD[metrics[i]],resultsMG[metrics[i]],resultsVP[metrics[i]],resultsNF[metrics[i]]]).T,columns=["SD", "MG", "VP", "NF"])
        data.set_index(resultsSD["index"]*10)
        plot = sns.lineplot(data,ax=axes[i],legend=True)
        plot.set_title(metrics2[i])
    fig.supxlabel('Epoch')
    plt.savefig(f"plots/epochs_plots")
    plt.show()

In [8]:
plotEpochs(resultsSD_MSD,resultsMG_MSD,resultsVP_MSD,resultsNF_MSD)

In [9]:
def makeDataBarPlot(resultsSD, resultsMG, resultsVP, resultsNF):
    results = [resultsSD, resultsMG, resultsVP, resultsNF]
    priors = ["SD","MG","VP","NF"]
    metrics = ["NMI","ARI","KBET"]
    res = resultsSD_MSD
    val = []
    prior = []
    metric = []
    error = []
    for j in range(4):
        for k in range(3):
            val.append(results[j][f"{metrics[k]} Mean"].iloc[38])
            prior.append(priors[j])
            metric.append(metrics[k])
            error.append(results[j][f"{metrics[k]} SD"].iloc[38])
    data = {'Value': val,'Prior': prior,'Metric': metric,'error': error}
    return data

In [10]:
df = makeDataBarPlot(resultsSD_MSD,resultsMG_MSD,resultsVP_MSD,resultsNF_MSD)

In [11]:
dataframe = pd.DataFrame(df)
dfNMI = dataframe.loc[dataframe["Metric"]=="NMI"]
dfARI = dataframe.loc[dataframe["Metric"]=="ARI"]
dfKBET = dataframe.loc[dataframe["Metric"]=="KBET"]

In [12]:
fig, axes = plt.subplots(3,1,figsize=(4,15))
df = [dfNMI, dfARI, dfKBET]
lims = [(0.5,None),(0.4,None),(0.2,None)]
for i in range(3):
    plot = sns.barplot(x='Metric', y='Value', data=df[i], hue='Prior',ax=axes[i])
    plt.xticks(visible=False)
    x_coords = [p.get_x() + 0.5 * p.get_width() for p in plot.patches]
    y_coords = [p.get_height() for p in plot.patches]
    plot.errorbar(x=x_coords, y=y_coords, yerr=df[i]["error"], fmt="none", c="k")
    plot.set_ylim(lims[i])
    plot.set(xlabel=None)
    plot.set(ylabel=None)
    plot.set(title=df[i]['Metric'].iloc[0])
    plot.set(xticks=[])
    plot.legend(loc=4)
    
plt.savefig("plots/barplot")
plt.show()